# Lab: Aprendizaje por refuerzo multi-agente (y V)

Habíamos visto en WoLF (`10_wolf.ipynb`) como los dos enfoques _value-based_ y _policy-based_ se pueden utilizar juntos para ofrecer una mayor estabilidad de entrenamiento. Aunque WoLF facilita ciertas mejoras en el rendimiento, carece de un cálculo real de gradientes, lo que puede resultar en una convergencia más lenta o no subóptima.

En este notebook, daremos un paso más allá introduciendo los algoritmos Actor-Critic, una familia de métodos que eficazmente integran los enfoques value-based y policy-based mediante el uso de gradientes para la optimización de políticas. Uno de los representantes más destacados y eficientes de esta familia es el Advantage Actor-Critic (A2C).

### Algoritmo A2C

El algoritmo A2C utiliza dos componentes principales:

* Actor: se encarga de determinar la acción a tomar, basándose en una política parametrizada $\pi(a | s; \theta)$.
* Crítico: proporciona una evaluación (función de valor $V$ o $Q$, dependiendo del algoritmo) de las acciones realizadas por el actor, calculando el valor de estado o el valor del estado-acción, y genera un valor de error o ventaja (llamado, genéricamente, _advantage_).

En base a estos dos componentes, el algoritmo A2C calcula, tras cada episodio, las ventajas que ha obtenido el actor, con cada una de sus acciones, según el valor otorgado por el crítico. En base a estas ventajas, se calcula el gradiente de la política y se modifica el modelo de política. Tanto el cálculo de la ventaja como el cálculo del gradiente de política se pueden hacer de diferentes maneras, pero las más estándar son:

* Para el cálculo de la ventaja, se suele usar la diferencia temporal $G_t = r_t + \gamma * V(s_{t+1}) - V(s_t)$, donde $V$ es el modelo del crítico.
* Para el cálculo del gradiente, se suele usar la fórmula de pérdida $L(\theta)$ que se utiliza en REINFORCE y que hemos visto en teoría (tema 6 de teoría, p. 38 o notebook `05_reinforce.ipynb`), adaptada al cálculo de la ventaja del punto anterior: $-\frac{1}{T} \sum_t^{T} G_t \cdot log \pi(a_t | s_t; \theta) $

La motivación principal es intentar mejorar la estabilidad al evitar que todo el aprendizaje se base en la modificación constante de un único valor (ya sea $V$, $Q$ o $\pi$), a partir del cual se obtiene el comportamiento entrenado. Ya hemos discutido en notebooks anteriores cómo DQN utiliza dos valores $Q$ para evitar este problema, o cómo WoLF permite salvar el problema de la no-estacionariedad al mantener, a la vez, una política, una política media y un valor Q. En el caso de los algoritmos Actor-Critic, la propuesta es tener un valor $Q$ o $V$, además de una política, interactuando constantemente.

En muchos entornos multiagente y de manera similar a como pasa con DQN y con similares limitaciones, los algoritmos Actor-Critic son adecuados para el aprendizaje por refuerzo, tanto en entornos puramente cooperativos como puramente competitivos. En situaciones de intereses mixtos la convergencia a un equilibrio no está garantizada.

El pseudocódigo del algoritmo DQN ([Multi-Agent Reinforcement Learning, Albrecht et al. 2024](https://www.marl-book.com/download/), Sección 8.2.5, p. 205) es:

<div>
<img src="a2c.png" width="500"/>
</div>

## Instalación y configuración

### Dependencias

In [ ]:
!pip --quiet install rlcard pettingzoo seaborn matplotlib numpy pandas tinynn pygame

### Imports necesarios

In [ ]:
import abc
import itertools
import random
import copy

from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tinynn.core.layer import Dense, ReLU
from tinynn.core.loss import MSE
from tinynn.core.model import Model
from tinynn.core.net import Net
from tinynn.core.optimizer import SGD

### Código de anteriores notebooks

In [ ]:
def pretty_print_array(ar):
    return np.array_str(np.array(ar), precision=2, suppress_small=True)

def draw_history(history, title):
    data = pd.DataFrame({'Episode': range(1, len(history) + 1), title: history})
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='Episode', y=title, data=data)

    plt.title(title + ' Over Episodes')
    plt.xlabel('Episode')
    plt.ylabel(title)
    plt.grid(True)
    plt.tight_layout()

    plt.show()

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def obs_to_state(obs):
    return obs.flatten()

In [ ]:
class GameModel:
    def __init__(self, num_agents, num_actions):
        self.num_agents = num_agents
        self.num_actions = num_actions
        self.action_space = self.generate_action_space()
        self.action_space_index = {joint_action: idx for idx, joint_action in enumerate(self.action_space)}

    def generate_action_space(self):
        actions_by_players = []
        for agent_id in range(self.num_agents):
            actions_by_players.append(range(self.num_actions))
        all_joint_actions = itertools.product(*actions_by_players)
        return [tuple(l) for l in all_joint_actions]

In [ ]:
class MARLAlgorithm(abc.ABC):
    @abc.abstractmethod
    def learn(self, joint_action, rewards, state, next_state, terminated, truncated):
        pass

    @abc.abstractmethod
    def select_action(self, state):
        pass

## Algoritmo A2C

En la constructora podéis ver los componentes principales:

* El modelo de actor: una red neuronal con tantas salidas como acciones (política $\pi$).
* El modelo de crítico: una red neuronal con una salida (valor de estado $V$).
* Un buffer donde almacenar todas las experiencias del episodio activo (como en REINFORCE).

In [ ]:
class A2C(MARLAlgorithm):
    def __init__(self, agent_id, game: GameModel,
                 gamma=0.95, alpha_actor=0.5, alpha_critic=0.5, epsilon=0.2, seed=42):
        self.agent_id = agent_id
        self.game = game
        self.alpha_actor = alpha_actor
        self.alpha_critic = alpha_critic
        self.gamma = gamma
        self.epsilon = epsilon
        self.rng = random.Random(seed)

        net_actor = Net([Dense(64), ReLU(),
                         Dense(32), ReLU(),
                         Dense(self.game.num_actions)])
        optimizer_actor = SGD(lr=self.alpha_actor)
        net_critic = Net([Dense(64), ReLU(),
                          Dense(32), ReLU(),
                          Dense(1)])
        optimizer_critic = SGD(lr=self.alpha_critic)
        self.actor = Model(net=net_actor, loss=MSE(), optimizer=optimizer_actor)
        self.critic = Model(net=net_critic, loss=MSE(), optimizer=optimizer_critic)
        self.episode_buffer = []
        self.metrics = {"actor_loss": [], "critic_loss": [], "advantage": []}

El método `compute_advantages` calcula la ventaja para una trayectoria completa, generalmente un episodio terminado. Como en REINFORCE, en este caso se usan las diferencias temporales de cada paso y se calculan de último a primer paso para ir acumulando los descuentos:

In [ ]:
class A2C(A2C):
    def compute_advantages(self, rewards, values, next_values, terminated):
        advantages = np.zeros_like(rewards)
        for t in reversed(range(len(rewards))):
            if terminated[t]:
                advantages[t] = rewards[t] - values[t]
            else:
                advantages[t] = rewards[t] + self.gamma * next_values[t] - values[t]
        return advantages

El método `update_actor_and_critic` se llama al final de cada episodio y se encarga de entrenar los dos modelos de actor y crítico:

* En primer lugar se obtienen los valores ($V(s_t)$) y los siguientes valores ($V(s_{t+1})$) para cada paso $t$.
* A partir de estos valores y la recompensa, se calculan las ventajas $G_t$.
* Luego se actualiza el crítico, calculando los gradientes en base a los valores originales más la ventaja correspondiente a cada paso.
* Finalmente se actualiza el actor en base a la fórmula de gradiente de política vista en REINFORCE: $-G_t \cdot log(\pi(a_t | s_t))$.

In [ ]:
class A2C(A2C):
    def update_actor_and_critic(self):
        states = np.array([e[0] for e in self.episode_buffer])
        actions = np.array([e[1] for e in self.episode_buffer])
        rewards = np.array([e[2] for e in self.episode_buffer])
        next_states = np.array([e[3] for e in self.episode_buffer])
        terminated = np.array([e[4] for e in self.episode_buffer])

        values = self.critic.forward(states)
        next_values = self.critic.forward(next_states)
        advantages = self.compute_advantages(rewards, values.flatten(), next_values.flatten(), terminated)

        # Actualizar crítico
        target_values = values + [[a] for a in advantages]
        critic_loss, critic_grads = self.critic.backward(values, target_values)
        self.critic.apply_grads(critic_grads)

        # Actualizar actor
        policies = [softmax(policy) for policy in self.actor.forward(states)]
        target_policies = copy.deepcopy(policies)
        for t in range(len(self.episode_buffer)):
            G_t = advantages[t]
            action_probs = policies[t]
            policy_gradient = -G_t * np.log(action_probs[actions[t]])
            target_policies[t][actions[t]] += policy_gradient
        actor_loss, actor_grads = self.actor.backward(np.array(policies), np.array(target_policies))
        self.actor.apply_grads(actor_grads)

        return actor_loss, critic_loss, np.mean(advantages)

El método de aprendizaje añade la experiencia al buffer del episodio. Si todos los agentes han terminado o han sido cortados (e.g. por tiempo), entonces entrenamos y vaciamos el buffer.

In [ ]:
class A2C(A2C):
    def learn(self, joint_action, rewards, state, next_state, terminated, truncated):
        experience = (state[self.agent_id], joint_action[self.agent_id],
                      rewards[self.agent_id], next_state[self.agent_id],
                      terminated[self.agent_id])
        self.episode_buffer.append(experience)
        if all([terminated[i] or truncated[i] for i in range(len(terminated))]):
            actor_loss, critic_loss, advantage = self.update_actor_and_critic()
            self.episode_buffer = []
            # Guardamos ventaja acumulada y las pérdidas de actor y crítico
            self.metrics['actor_loss'].append(abs(actor_loss))
            self.metrics['critic_loss'].append(abs(critic_loss))
            self.metrics['advantage'].append(advantage)

Por último, la función que representa a la política es diferente a la de Q-Learning. En gradiente de política no hay $\epsilon$-greediness ya que la política permite exploración siempre y cuando los valores no converjan abruptamente a 1s y 0s:

In [ ]:
class A2C(A2C):
    def select_action(self, state):
        np.random.seed(self.rng.randint(0, 10000))
        return np.random.choice(range(self.game.num_actions), p=softmax(self.actor.forward(np.array([state]))[0]))

## Entrenamiento

In [ ]:
def train_agents(env, scenarios, epochs, gammas, alphas, epsilons, seeds):
    env.reset()
    agent_strings = env.agents
    num_agents = len(env.agents)

    # Crear modelo de juego y algoritmos para cada agente
    game_model = [GameModel(num_agents=num_agents, num_actions=env.action_space(agent_strings[i]).n)
                  for i in range(num_agents)]
    algorithms = [A2C(agent_id, game_model[agent_id], gamma=gammas[agent_id],
                      alpha_actor=alphas[agent_id], alpha_critic=alphas[agent_id],
                      epsilon=epsilons[agent_id],
                      seed=seeds[agent_id])
                  for agent_id in range(num_agents)]

    # Métricas a guardar: recompensas y acciones
    cumulative_rewards = [[0] * num_agents]
    actions_played = [[]] * num_agents

    for _ in tqdm(range(epochs)): # Recorridos completos a todos los escenarios
        for idx in range(scenarios): # Escenarios aleatorios
            # Observación inicial
            observations, infos = env.reset(seed=idx)
            states = [obs_to_state(observations[agent_strings[i]]) for i in range(num_agents)]
            episode_rewards = [0] * num_agents
            while env.agents:
                env.render()

                # Selección de acciones
                actions = []
                petting_zoo_actions = {}
                for i in range(num_agents):
                    actions.append(algorithms[i].select_action(states[i]))
                    actions_played[i].append(actions[i])
                    petting_zoo_actions[agent_strings[i]] = actions[i]

                # Actualización de entorno
                observations, rewards, terminations, truncations, infos = env.step(petting_zoo_actions)

                next_states = []
                indexed_rewards = []
                indexed_terminations = []
                indexed_truncations = []
                for i in range(num_agents):
                    # Conversión de formato petting zoo -> lista
                    next_states.append(obs_to_state(observations[agent_strings[i]]))
                    indexed_rewards.append(rewards[agent_strings[i]])
                    indexed_terminations.append(terminations[agent_strings[i]])
                    indexed_truncations.append(truncations[agent_strings[i]])
                    episode_rewards[i] += indexed_rewards[i]
                for i in range(num_agents):
                    # Entrenamiento
                    algorithms[i].learn(actions, indexed_rewards, states, next_states,
                                        indexed_terminations, indexed_truncations)
                states = next_states
            cumulative_rewards.append([cumulative_rewards[-1][i] + indexed_rewards[i]
                                       for i in range(num_agents)])
            env.close()
    return game_model, algorithms, cumulative_rewards, actions_played

In [ ]:
from pettingzoo.sisl import pursuit_v4

env = pursuit_v4.parallel_env(render_mode="rgb_array", x_size=8, y_size=8,
                              n_evaders=1, n_pursuers=2, surround=False,
                              obs_range=2, max_cycles=50)
env.reset()
num_agents = len(env.agents)
game_model, algorithms, cumulative_rewards, actions_played = \
    train_agents(env, scenarios=10, epochs=10, gammas=[0.95] * num_agents, alphas=[0.1] * num_agents,
                 epsilons=[0.2] * num_agents, seeds=[i for i in range(num_agents)])

print(f"Recompensas acumuladas: {cumulative_rewards[-1][0]}, {cumulative_rewards[-1][1]}")

In [ ]:
draw_history([cumulative_rewards[idx][0] for idx in range(len(cumulative_rewards))], "Rewards")
draw_history(algorithms[0].metrics['advantage'], "Advantage")
draw_history(algorithms[0].metrics['actor_loss'], "Actor loss")
draw_history(algorithms[0].metrics['critic_loss'], "Critic loss")


## Evaluación

In [ ]:
env = pursuit_v4.parallel_env(render_mode="human", x_size=8, y_size=8,
                              n_evaders=1, n_pursuers=2, surround=False,
                              obs_range=2, max_cycles=1000)
observations, infos = env.reset(seed=random.randint(0, 10000))
agent_strings = env.agents
all_rewards = [0]

states = [obs_to_state(observations[agent_strings[i]]) for i in range(num_agents)]
while env.agents:
    env.render()
    actions = [algorithms[i].select_action(states[i]) for i in range(num_agents)]
    petting_zoo_actions = {agent_strings[i]: actions[i] for i in range(num_agents)}
    observations, rewards, terminations, truncations, infos = env.step(petting_zoo_actions)
    next_states = [obs_to_state(observations[agent_strings[i]]) for i in range(num_agents)]
    indexed_rewards = [rewards[agent_strings[i]] for i in range(num_agents)]
    indexed_terminations = [terminations[agent_strings[i]] for i in range(num_agents)]
    all_rewards.append(all_rewards[-1] + sum(indexed_rewards))
    states = next_states
if any(indexed_terminations):
    print("Todos los evaders han sido cazados!")
env.close()
draw_history(all_rewards, "Cumulative reward")